In [18]:
# https://www.bitaddress.org/
# https://en.wikipedia.org/wiki/Base58

BASE58_CHARS = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'


def map_base58():
    '''Create a map of chars for Base58'''
    tmp_dir = {}
    i = 0
    for c in BASE58_CHARS:
        tmp_dir[c] = i
        i = i + 1
    return tmp_dir


BASE58_MAP = map_base58()


def base58_enc(in_num):
    '''Encode a number into Bitcoin Base58 format'''
    if in_num < 0:
        raise Exception("Positive integers only")
    tmp_out = []
    while in_num > 0:
        tmp_out.append(BASE58_CHARS[in_num % 58])
        in_num = in_num // 58
    tmp_out.reverse()
    return ''.join(tmp_out)


def base58_dec(in_str):
    '''Decode a Bitcoin Base58 string to a number'''
    out_num = 0
    for c in in_str:
        if c not in BASE58_MAP:
            raise Exception("Invalid char, not in Base58: %c" % c)
        out_num = out_num*58 + BASE58_MAP[c]
    return out_num

In [19]:
# For testing
PRIV_KEY_BASE58 = 'L3BQRZyUzNPUPbt1HtGby9UwVb5iz2RyEk9jQk1vqhnL5CwFZHiX'
BITCOIN_ADDR_BASE58 = '1D2Gme2513ncWsxB4DchzT3ukeNUXYVv3c'

print(base58_enc(123))
print(base58_dec('38'))
print(base58_dec(PRIV_KEY_BASE58))
print(base58_dec(BITCOIN_ADDR_BASE58))

38
123
16384730769480552998872844202963833909082627022173914913578359054339044055507470509937806088
3233478352685485411235947024398804356817507813717140601691


In [70]:
# https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses
# https://github.com/warner/python-ecdsa
# https://docs.python.org/3/library/hashlib.html
# Terminal: openssl ecparam -list_curves | grep -i secp256k1

import struct, hashlib
from ecdsa import SigningKey, SECP256k1
from binascii import hexlify, unhexlify

# Step 0: Having a private ECDSA key
priv_key_hex = '18E14A7B6A307F426A94F8114701E7C8E774E7F9A47E2C2035DB29A206321725'
priv_key = int(priv_key_hex, 16)
secexp = priv_key
signing_key = SigningKey.from_secret_exponent(secexp, curve=SECP256k1)
sk_hex = hexlify(signing_key.to_string())
print('Signing key:\n%s\n' % sk_hex)

# Step 1: Take the corresponding public key generated with it
# (65 bytes, 1 byte 0x04, 32 bytes corresponding to X coordinate,
# 32 bytes corresponding to Y coordinate)
vk = signing_key.get_verifying_key()
vk_hex = hexlify(vk.to_string())
print('Verifying key:\n%s\n' % vk_hex)
full_pub_key_hex = '04%s' % vk_hex.decode('utf-8')
full_pub_key = unhexlify(full_pub_key_hex)
print('Full format public key:\n%s\n' % full_pub_key_hex)
assert len(full_pub_key) == 65

# Step 2: Perform SHA-256 hashing on the public key
sha256_hasher = hashlib.sha256(full_pub_key)
print(sha256_hasher.hexdigest())

# Step 3: Perform RIPEMD-160 hashing on the result of SHA-256
ripemd160_hasher = hashlib.new('ripemd160')
ripemd160_hasher.update(sha256_hasher.digest())
print(ripemd160_hasher.hexdigest())

# Step 4: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)
mainnet_format_hex = '00%s' % ripemd160_hasher.hexdigest()
print(mainnet_format_hex)
mainnet_format = unhexlify(mainnet_format_hex)

# Step 5: Perform SHA-256 hash on the extended RIPEMD-160 result
sha256_hasher = hashlib.sha256(mainnet_format)
print(sha256_hasher.hexdigest())

# Step 6: Perform SHA-256 hash on the result of the previous SHA-256 hash
sha256_hasher2 = hashlib.sha256(sha256_hasher.digest())
print(sha256_hasher2.hexdigest())

# Step 7: Take the first 4 bytes of the second SHA-256 hash. This is the address checksum
first_4bytes = sha256_hasher2.digest()[:4]

# Step 8: Add the 4 checksum bytes from stage 7 at the end of
# extended RIPEMD-160 hash from stage 4.
# This is the 25-byte binary Bitcoin Address.
bitcoin_addr_raw = mainnet_format + first_4bytes
assert len(bitcoin_addr_raw) == 25
bitcoin_addr_hex = hexlify(bitcoin_addr_raw)
print(bitcoin_addr_hex)

# Convert the result from a byte string into a base58 string using Base58Check encoding.
# This is the most commonly used Bitcoin Address format
bitcoin_addr_num = int(bitcoin_addr_hex, 16)
bitcoin_addr = base58_enc(bitcoin_addr_num)
print(bitcoin_addr)

Signing key:
b'18e14a7b6a307f426a94f8114701e7c8e774e7f9a47e2c2035db29a206321725'

Verifying key:
b'50863ad64a87ae8a2fe83c1af1a8403cb53f53e486d8511dad8a04887e5b23522cd470243453a299fa9e77237716103abc11a1df38855ed6f2ee187e9c582ba6'

Full format public key:
0450863ad64a87ae8a2fe83c1af1a8403cb53f53e486d8511dad8a04887e5b23522cd470243453a299fa9e77237716103abc11a1df38855ed6f2ee187e9c582ba6

600ffe422b4e00731a59557a5cca46cc183944191006324a447bdb2d98d4b408
010966776006953d5567439e5e39f86a0d273bee
00010966776006953d5567439e5e39f86a0d273bee
445c7a8007a93d8733188288bb320a8fe2debd2ae1b47f0f50bc10bae845c094
d61967f63c7dd183914a4ae452c9f6ad5d462ce3d277798075b107615c1a8a30
b'00010966776006953d5567439e5e39f86a0d273beed61967f6'
6UwLL9Risc3QfPqBUvKofHmBQ7wMtjvM


In [42]:
from ecdsa import SigningKey, NIST384p
sk = SigningKey.generate(curve=NIST384p)
sk_string = sk.to_string()
sk2 = SigningKey.from_string(sk_string, curve=NIST384p)

print(sk_string)
print(sk)
print(sk2)

b'\x1a\xf8\x9c\xc8.\xbe\x84\xffpM\xaa\xa0\xf6\x06\xafS\x05\xda|\xaa\x92\xadd\x8b\xce\xc7z\xfa\xaa0u\x9c\x93N\xec\x8as\xdb\xfb\xebs\xfdd~NA*\xc5'
